<a href="https://colab.research.google.com/github/ma850419/Various_scripts/blob/main/ayrclass2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='velvety-ring-328419')

In [ ]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

In [ ]:
#imageROI =ee.FeatureCollection('users/mohamadawadlebanon/borderutm36n')
imageROI =ee.FeatureCollection('users/mohamadawadlebanon/Iranborder')
#trainingPolys = ee.FeatureCollection('users/mohamadawadlebanon/fieldsamples_2019_36N')
#evalPolys=ee.FeatureCollection('users/mohamadawadlebanon/fieldsamples_2019_36N')
training = ee.FeatureCollection('users/mohamadawadlebanon/crop2')

In [ ]:
import folium

# Initialize Earth Engine

# Use Sentinel-2 surface reflectance data.
s2sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Cloud masking function for Sentinel-2.
def maskS2sr(image):
    qa = image.select('QA60')  # Sentinel-2 cloud mask band
    mask = qa.eq(0)  # Keep only pixels where QA60 is clear
    return image.updateMask(mask).divide(10000)  # Scale reflectance values

# Clip function
def clipimage(image):
    return image.clip(imageROI)

# Define Image ROI (Replace with a valid region)
#imageROI = ee.Geometry.Rectangle([35, 30, 40, 35])  # Example ROI

# Create a cloud-masked median composite for July 2019
image = s2sr.filterDate('2024-05-01', '2024-05-31').map(clipimage).map(maskS2sr).median()
print(image.getInfo())
# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})  # RGB composite
map = folium.Map(location=[28, 53], zoom_start=6)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Sentinel-2 Composite',
).add_to(map)

'''mapid = image.getMapId({'bands': ['B11'], 'min': 0, 'max': 0.3})  # Thermal (SWIR Band)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='SWIR Band (B11)',
).add_to(map)

map.add_child(folium.LayerControl())'''

# Display map
map


In [ ]:
image_np = geemap.ee_to_numpy(image, imageROI, bands=['B4', 'B3', 'B2'], scale=55)  # 10m resolution


In [ ]:
import geemap
import numpy as np

# Extract image bands
region = imageROI  # Define a bounding box (Iran region)

# Convert to NumPy with specified region
image_np = geemap.ee_to_numpy(image,imageROI, bands=['B4', 'B3', 'B2'], scale =60)

'''training_image = training.reduceToImage(
    properties=['class'],
    reducer=ee.Reducer.first()
)'''
samples_image = training.reduceToImage(['class'], ee.Reducer.mode())  # Convert points to raster
# Change 'class' to 'mode' to match the band name created by reduceToImage
samples_np = geemap.ee_to_numpy(samples_image, region=region, bands=['mode'], scale =60)  # Extract pixels

# Define region (Ensure it's bounded)

# Convert to NumPy
#samples_np = geemap.ee_to_numpy(training_image, region=region, scale=40)

In [ ]:
#image_np = np.expand_dims(image_np, axis=-1)  # Converts to (height, width, 1)
image_np = np.expand_dims(image_np, axis=0)  # Becomes (1, 1439, 1825, 3)
samples_np = np.expand_dims(samples_np, axis=0)  # Becomes (1, 1439, 1825, 1)

print(image_np.shape)
#print(image_np)  # Inspect sample values

print(samples_np.shape )

In [ ]:
new_height = image_np.shape[1] - (image_np.shape[1] % 2)  # Ensure divisible by 2
new_width = image_np.shape[2] - (image_np.shape[2] % 2)

image_np = image_np[:, :new_height, :new_width, :]  # Trim image size
samples_np = samples_np[:, :new_height, :new_width, :]  # Match target size
print(image_np.shape)
#print(image_np)  # Inspect sample values

print(samples_np.shape )

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D

# Define U-Net Model
model = tf.keras.Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(image_np.shape[1], image_np.shape[2], image_np.shape[3])),
    MaxPooling2D((2, 2)),
    UpSampling2D((2, 2)),
    Conv2D(1, (1, 1), activation='sigmoid', padding='same')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model (using GEE sample data)
model.fit(image_np, samples_np, epochs=10, batch_size=16)


In [ ]:
predictions = model.predict(image_np)
import numpy as np
predictions= np.squeeze(predictions)
# Rescale predictions to class values 1–20
num_classes = 20
predictions_scaled = np.round(predictions * num_classes).astype(np.uint8)  # Convert float to integer class labels
predictions_scaled = np.clip(predictions_scaled, 1, num_classes)  # Ensure values stay within 1-20

# Define region where image should be placed

# Convert NumPy array into an Earth Engine image

# Print details to verify conversion
#print(predicted_image.getInfo())

# Convert to EE Image
predicted_image = geemap.numpy_to_ee(predictions_scaled, imageROI)
'''task = ee.batch.Export.image.toAsset(
    image=predicted_image,
    description="UNet_Prediction",
    assetId="users/mohamadawadlebanon/UNet_Prediction",
    scale=60, # Specify the scale as done during the numpy conversion
    region=imageROI.geometry() # Specify the region for export
)'''
task = ee.batch.Export.image.toDrive(
    image=predicted_image,
    description="UNet_Prediction",
    #assetId="users/mohamadawadlebanon/UNet_Prediction",
    scale=1000, # Specify the scale as done during the numpy conversion
    region=imageROI.geometry() # Specify the region for export
)
task.start()

print(f"Export task started: {task.status()}") # Print task status

In [ ]:
viz_params = {
    'min': 1,
    'max': 20,
    'palette': [
        'red', 'yellow', 'green', 'blue', 'purple', 'orange', 'brown', 'cyan', 'pink',
        'magenta', 'lime', 'gray', 'black', 'white', 'gold', 'navy', 'teal', 'maroon',
        'turquoise', 'violet'
    ]
}


In [ ]:
predicted_image = predicted_image.setDefaultProjection('EPSG:4326', None, 30)  # WGS84 with 30m scale



In [ ]:
print(predictions_scaled.shape)
  # Ensure it prints valid geometric coordinates
pixels = ee.Array(predictions_scaled.tolist())  # Convert NumPy array to EE array
predicted_image = ee.Image(pixels).clip(imageROI)  # Clip to the target region


In [ ]:
print(predicted_image.projection().getInfo())  # Confirms raster properties


In [ ]:
print(predicted_image)



In [ ]:
Map.addLayer(predicted_image, {'min': 1, 'max': 20, 'palette': ['red', 'blue', 'green']}, "Predicted Image")


In [ ]:
Map = geemap.Map()
Map.addLayer(predictions_scaled, viz_params, "U-Net Classified Image")
Map.centerObject(region, 10)  # Adjust zoom level
Map


In [ ]:
predicted_image_ee = geemap.numpy_to_ee(predictions_scaled, imageROI)

# Set the projection and scale explicitly
predicted_image_ee = predicted_image_ee.setDefaultProjection(
    crs='EPSG:4326',  # Specify a common projection
    scale=60          # Use the same scale as when converting to numpy
)


Map = geemap.Map()
# Add the Earth Engine image to the map
Map.addLayer(predicted_image_ee, {}, "U-Net Classified Image")
Map.centerObject(imageROI.geometry(), 10)  # Adjust zoom level and use the region's geometry
Map